For FGDC and ISO, can we identify the age of the metadata record?

What's the oldest? What's the age distribution?

For this, we're going to access the RDS.

for iso, on root node:

```
<gmd:dateStamp>
        <gco:Date>2015-09-10</gco:Date>
    </gmd:dateStamp>
```



for fgdc:
```
metainfo/metd (released)
metainfo/metrd (reviewed)
```

In [19]:
import json as js  # name conflict with sqla
import sqlalchemy as sqla
from sqlalchemy.orm import sessionmaker
from sqlalchemy.dialects.postgresql import *
from sqlalchemy import and_
from semproc.xml_utils import *
from mpp.models import Response
from datetime import datetime
from lxml import etree

In [6]:
def _convert_date(datestr):
    # convert the 4-8 char to an iso date and deal with unknowns
    if datestr.lower() in ['unknown', 'none']:
        return ''

    year = datestr[:4]
    month = datestr[4:6] if len(datestr) > 4 else '1'
    day = datestr[6:] if len(datestr) > 6 else '1'
    try:
        d = datetime(int(year), int(month), int(day))
        return d.isoformat()
    except:
        return ''

In [ ]:
# load the postgres connection file
with open('../local/big_rds.conf', 'r') as f:
    conf = js.loads(f.read())

# our connection
engine = sqla.create_engine(conf.get('connection'))
Session = sessionmaker()
Session.configure(bind=engine)
session = Session()

In [13]:
# just for resets
session.rollback()

In [43]:
# TODO: modify this to get iso and fgdc xml only
# clauses = [Response.format == 'xml']

sketchy_sql = '''select distinct source_url_sha, source_url, cleaned_content
from (
  select source_url_sha, source_url, cleaned_content, unnest(schemas) schs
  from responses
  where format = 'xml'
) a
where schs ilike '%fgdc%' or ((schs ilike '%gmi%' or schs ilike '%gmd%') and source_url not ilike '%csw%' and source_url not ilike '%oai%')
;'''

result = session.execute(sketchy_sql)

In [ ]:
timestamps = []
for r in result:
    print r['source_url']
    
    xml = etree.fromstring(r['cleaned_content'].encode('utf-8'))
    
    md_date = extract_item(xml, ['//*', 'dateStamp', 'Date'])
    
    if not md_date:
        md_date = extract_item(xml, ['//*', 'metainfo', 'metd'])
        if not md_date:
            continue
            
    timestamps.append((r['source_url_sha'], r['source_url'], md_date))
    

In [45]:
len(timestamps)

3085

In [48]:
min(timestamps, key=lambda x: x[2] if x[2] != '1900-01-01' else datetime.now().isoformat())

(u'dc8c43ff562b571843eea607cd793288c0a1455c',
 u'https://catalog.data.gov/harvest/object/59da2b52-3ce8-4da7-90cc-b1e178488d87',
 '1990-01-01')